# Combining different spatial resoultions

This Notebook can be used to combine different spatial resoultions and their annotations.

Target:

* One annotation file
* One "images" folder
  * Multiple Subfolders

Strategy:

* Load all annotation files
* Take a part of the file name
* Create images subfoler with file name
* Write all annotations to one file
* rename the files and the image ID in the final folder

## How to use

You can edit the file path information in the cell labeled *User Settings*.

In [ ]:
# libraries
import os
import json
import datetime
from pycocotools.coco import COCO

In [ ]:
# User Settings

# file path for the original annoation file as a list object
annotation_files = ["./annotations/instances_default.json"]

# path to the scaled annoation files
scaled_annotation_files = './scaled_coco_annotations'

# # This is where you want the combined annotations saved.
combined_annotation_files = './combined_coco_annotations'

In [ ]:
def get_filename(path: str):
    """Get the basefile name form a file path

    Args:
        path (str): _description_

    Returns:
        str: Name of the file
    """
    # Normalize the path to use the correct separator for the current OS
    normalized_path = os.path.normpath(path)
    # Extract the file name
    filename = os.path.basename(normalized_path)
    return filename

def merge_coco_json(json_files: list, output_file: str):
    """Merge and write COCO annotation files

    Args:
        json_files (list): List of all json files which contain annotations
        output_file (str): Tragte file for the combined annotations
    """
    merged_annotations = {
        "info": {},
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }

    image_id_offset = 0
    category_id_offset = 0
    existing_category_ids = set()

    for idx, file in enumerate(json_files):
        coco = COCO(file)

        # create subfolder for the images
        folder_name = get_filename(file).split(".")[0].split("_")[-4:]
        folder_name = "_".join(folder_name)
        os.makedirs(os.path.join("train_images", folder_name), exist_ok=True)
        
        # get scale value from file name
        details_list = get_filename(file).split(".")[0].split("_")
        print(details_list)
        if "scaled" in details_list:
            # get the position of "scaled" word and then take the value: *_scaled_n_*
            pos = details_list.index("scaled")
            scale = float(details_list[pos+1])
        
        else:
            # defualt scale to "1"
            scale = 1

        # Update image IDs to avoid conflicts
        for image in coco.dataset['images']:
            image['id'] += image_id_offset
            image["height"] = int(image["height"]/scale)
            image["width"] = int(image["width"]/scale)
            image['file_name'] = f"{folder_name}/{image['file_name']}"
            merged_annotations['images'].append(image)

        # Update annotation IDs to avoid conflicts
        for annotation in coco.dataset['annotations']:
            #annotation['id'] += annotation_id_offset
            annotation['image_id'] += image_id_offset
            merged_annotations['annotations'].append(annotation)

        # Update categories and their IDs to avoid conflicts
        for category in coco.dataset['categories']:
            if category['id'] not in existing_category_ids:
                category['id'] += category_id_offset
                merged_annotations['categories'].append(category)
                existing_category_ids.add(category['id'])

        # claculate offsets for the next loop
        image_id_offset = len(merged_annotations['images'])
        category_id_offset = len(merged_annotations['categories'])

    # Save merged annotations to output file
    with open(output_file, 'w') as f:
        json.dump(merged_annotations, f)


In [ ]:
# Combining the annotation files from different resoltuions
for annotation_file in annotation_files:
    # gte filie name
    file_name = get_filename(annotation_file).split(".")[0]
    # get corresponding scaled files
    scaled_files = [os.path.join(scaled_annotation_files, file) for file in os.listdir(scaled_annotation_files)]
    scaled_files.append(annotation_file)

    # TODO: ADD/EDIT names of images coco_data["images"]

    # List of paths to COCO JSON files to merge
    json_files = scaled_files

    # Output file path for merged annotations
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    output_file = get_filename(annotation_file)
    output_file = f'{current_time}_{output_file}'.replace(':', '-').replace(' ', '_')
    output_file = os.path.join(combined_annotation_files, output_file)

    # Merge COCO JSON files
    merge_coco_json(json_files, output_file)

    print("Merged COCO JSON files saved to", output_file)
